In [55]:
# Automatically keep reloading imported modules
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
empty_notebooks_dir_path = 'tac_notebooks'
log_dir_filepath = 'tac_raw_logs'

In [57]:

import os
from _log_parser import LogParser
from _nb_parser import NotebookParser
from _utils import get_all_file_with_extension_in_dir_recursively

all_log_filepathes = get_all_file_with_extension_in_dir_recursively(log_dir_filepath, ".log")
all_log_filepathes.sort()
print(f'There are {len(all_log_filepathes)} log files in {log_dir_filepath} directory')
# for i, log_filepath in enumerate(all_log_filepathes):
#     parsed_log = LogParser(log_filepath).parse()
#     print(f"{i} {log_filepath}")

all_training_notebooks_filepathes = {
    os.path.basename(nb_filepath): nb_filepath
    for nb_filepath in
    get_all_file_with_extension_in_dir_recursively(empty_notebooks_dir_path, ".ipynb")
}
print(f'\nThere are total {len(all_training_notebooks_filepathes)} training notebooks')
# for i, (_, nb_filepath) in enumerate(all_training_notebooks_filepathes.items()):
#     print(f"{i} {nb_filepath}")

print('\nExample selected log file:')
selected_log_filepath = all_log_filepathes[1]
print(f'Selected log file: {selected_log_filepath}')
log_parser = LogParser(selected_log_filepath).parse()
log_parser_per_notebook = log_parser.attach_notebooks(all_training_notebooks_filepathes, verbose=False)
print(f'\nLog parser per notebook: {log_parser_per_notebook}')


There are 16 log files in tac_raw_logs directory

There are total 230 training notebooks

Example selected log file:
Selected log file: tac_raw_logs/subject-0-logs/knic-tac-evaluation.log
There are 7 notebooks with logs
There are 5 out of 7 notebooks with continous logs

Log parser per notebook: {'X-subject-0-1-companion-geonames.ipynb': <_log_parser.LogParser object at 0x7fdcf063c820>, 'X-subject-0-11-companion-ICT.ipynb': <_log_parser.LogParser object at 0x7fdcf07cf8e0>, 'X-subject-0-13-companion-ICT.ipynb': <_log_parser.LogParser object at 0x7fdcf07ba610>, 'X-subject-0-5-companion-geonames.ipynb': <_log_parser.LogParser object at 0x7fdcf80294c0>, 'X-subject-0-9-companion-MT.ipynb': <_log_parser.LogParser object at 0x7fdcf06f5730>}


In [58]:
for nb_filepath, nb_log_parser in log_parser_per_notebook.items():
    selected_nb_filepath = all_training_notebooks_filepathes[nb_filepath]
    nb_parser = NotebookParser(selected_nb_filepath)
    print(f'Selected notebook: {selected_nb_filepath} with {len(nb_parser)} cells in the selected notebook')
    print(f'Opened notebook: {nb_parser.filepath}')
    print(f'Log Parser only notebook: {nb_log_parser.filepath}')
    
print('>> Debug')
print(nb_parser.filepath)
print(nb_log_parser.filepath)
print(nb_parser[1])
print(nb_log_parser.get_only(entry_type='CELL_SELECTED')[1])
    

Selected notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-1-companion-geonames.ipynb with 8 cells in the selected notebook
Opened notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-1-companion-geonames.ipynb
Log Parser only notebook: tac_raw_logs/subject-0-logs/knic-tac-evaluation.log
Selected notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-11-companion-ICT.ipynb with 11 cells in the selected notebook
Opened notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-11-companion-ICT.ipynb
Log Parser only notebook: tac_raw_logs/subject-0-logs/knic-tac-evaluation.log
Selected notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-13-companion-ICT.ipynb with 9 cells in the selected notebook
Opened notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-13-companion-ICT.ipynb
Log Parser only notebook: tac_raw_logs/subject-0-logs/knic-tac-evaluation.log
Selected notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-5-companion-ge

In [59]:
from _nb_progress import get_notebook_progress
nb_progress = get_notebook_progress(nb_parser, nb_log_parser)


In [69]:
for step in nb_progress:
    print()
    step.reset()
    for change in step:
        print(change)
    step.reset()


[
    {
        "cell_type": "markdown",
        "id": 0,
        "source": [
            "Sample aligned data"
        ]
    },
    {
        "cell_type": "code",
        "id": 1,
        "source": [
            "src_lines = [\"I would like to live in Berlin after I retire.\",",
            "             \"My friends and I plan to open a cheese shop.\",",
            "             \"We would also sell wine and beer if it proved profitable.\"]",
            "ref_lines = [\"Ich m\\xf6chte nach meiner Pensionierung in Berlin leben.\",",
            "             \"Meine Freunde und ich planen, einen K\\xe4seladen zu er\\xf6ffnen.\",",
            "             \"Wir w\\xfcrden auch Wein und Bier verkaufen, wenn es sich rentiert.\"]"
        ]
    },
    {
        "cell_type": "markdown",
        "id": 2,
        "source": [
            "Set up the Facebook English-to-German MT model"
        ]
    },
    {
        "cell_type": "code",
        "id": 3,
        "source": []
    },
    {
 

In [71]:
from _utils import (
    construct_prompt,
    prompt,
    pprint_msg,
    count_tokens_in_prompt_messages
)
import json
import openai
from tqdm import tqdm
from _log_parser import LogEntry
from _nb_progress import get_notebook_progress
from _utils import Tee


def inject_content(assistant_msg, step_cell_id: int, change: LogEntry):
    assistant_msg = assistant_msg.copy()
    try:
        prepared_json_format = json.loads(assistant_msg['content'])
    except:
        try:
            prepared_json_format = eval(assistant_msg['content'])
        except:
            prepared_json_format = assistant_msg['content']
            
    prepared_json_format[step_cell_id]['content'] = change.get_formatted_content()
    return {'content': prepared_json_format}


nb_progress = get_notebook_progress(nb_parser, nb_log_parser)

# remove no-op steps
nb_progress = [progress for progress in nb_progress if len(progress) > 0]

nb_parser_filename = os.path.basename(nb_parser.filepath)
nb_log_parser_filename = os.path.basename(nb_log_parser.filepath)
unique_log_name = f'{nb_parser_filename}_{nb_log_parser_filename}.description_seq'
tee = Tee(f'{unique_log_name}.txt')
def print(*args, **kwargs):
    with tee:
        return __builtins__.print(*args, **kwargs)

print(f'Notebook: {nb_parser.filepath}')
print(f'Log: {nb_log_parser.filepath}')
print(f'Number of progress steps: {len(nb_progress)}')
print(f'Number of progress steps unrolled: {sum([len(step) for step in nb_progress])}')

assistant_msgs = []
prev_steps_applied = []
for step_i, step in enumerate(nb_progress):
    step.reset()
    next_change_type = 'insert'
    # prev_msgs = [] # TODO should I reset prev_msgs upon each completed step?
    for change_i, nb_parser_with_change_applied in enumerate(step):
        # Use default prompt -- will be appended automatically by construct_prompt
        prev_msgs = []
        for prev_step, prev_response in zip(prev_steps_applied, assistant_msgs):
            prev_msgs.append({ "role": "user", "content": str(prev_step)})
            prev_msgs.append(prev_response)

        while True:
            try:
                prompt_msgs = construct_prompt(nb_parser_with_change_applied, prev_messages=prev_msgs)
                assistant_msgs.append(prompt(prompt_msgs))
                break
            except openai.error.InvalidRequestError as e:
                if len(prev_msgs) > 0:
                    prev_msgs = prev_msgs[-1:]
                else:
                    raise e
                
        prev_steps_applied.append(nb_parser_with_change_applied)
                
        print('><'*50)
        # print('prompt_msgs')
        # for i, prompt_msg in enumerate(prompt_msgs):
        #     print(i)
        #     pprint_msg(prompt_msg)
            
        print(f'Step {step_i} Change @ {step.cell_id}:: {change_i}:  {next_change_type}')
        next_change_type = 'update'
        print('START OF CHANGE')
        print(step.entries[change_i].print())
        print('END OF CHANGE')
        print('><'*50)
        
        print(f'Step {step_i} Change {change_i} Response:')
        print('START OF RESPONSE')
        injected_contented_assistant_msg = inject_content(assistant_msgs[-1], step.cell_id, step.entries[change_i])
        pprint_msg(injected_contented_assistant_msg)
        print('END OF RESPONSE')
        

Notebook: tac_notebooks/tac_notebooks/X-subject-0/X-subject-0-9-companion-MT.ipynb
Log: tac_raw_logs/subject-0-logs/knic-tac-evaluation.log
Number of progress steps: 4
Number of progress steps unrolled: 5
num_tokens from prompt: 667
num_tokens from response: 758
><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><
Step 0 Change @ 1:: 0:  insert
START OF CHANGE
╒════════════════╤══════════════════════════════════════════════════════════════════════════════════════╕
│       Entry ID │ 441                                                                                  │
├────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│     Entry Type │ CELL_EXECUTION_END                                                                   │
├────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│ Timestamp (ms) │ 1690240621753                           

### Testing LogParser class

In [ ]:

log_filepath = "tac_raw_logs/subject-0-logs/knic-tac-evaluation.log"
log_parser = LogParser(log_filepath).parse()

# cell_selected_entries = log_parser.get_entries_by_type("CELL_SELECTED")
# for entry in cell_selected_entries:
#     entry.print()

print('Notebooks:', log_parser.get_notebooks())
print('Entry types:', log_parser.get_entry_types())
print('Cell types:', log_parser.get_cell_types())
print('Users:', log_parser.get_users())
    
content_entries = log_parser.get_filtered(
    entry_type=[
        'CELL_EXECUTION_END', 
        'CODE_DESCRIPTION_SUGGEST'
    ]
)
print(len(content_entries))
for entry in content_entries:
    entry.print()


Notebooks: {'X-subject-0-11-companion-ICT.ipynb', 'X-subject-0-1-companion-geonames.ipynb', 'spacy-TA-C.ipynb', 'ICT-training.ipynb', 'geonames-correct-answers.ipynb', 'MT-correct-answers.ipynb', 'X-subject-0-5-companion-geonames.ipynb', 'X-subject-0-9-companion-MT.ipynb', 'X-subject-0-13-companion-ICT.ipynb', 'X-subject-0-3-companion-geonames.ipynb', 'X-subject-0-7-companion-MT.ipynb', 'MT-training.ipynb', 'geonames-training.ipynb'}
Entry types: {'CELL_SELECTED', 'NOTEBOOK_OPENED', 'TGM_UPDATE_TIME_SUBTRACT', 'CELL_EXECUTION_END', 'NOTEBOOK_CLOSED', 'CODE_DESCRIPTION_UPDATE', 'TGM_QUESTION_ANSWERED', 'STEP_WHAT_WHY_TIME', 'TASK_WHAT_WHY_TIME', 'CELL_EXECUTION_BEGIN', 'CODE_DESCRIPTION_SUGGEST', 'TGM_QUESTION_ASKED'}
Cell types: {'0.573807954788208', '0.9674360752105713', '4', '12', '0.7641811370849609', '15', 'code', '4.802643299102783', '20', '0.23778820037841797', '5.193816900253296', '0.6458990573883057', '30', 'markdown', '4.936611890792847', '0.8737890720367432', '1.0803887844085